In [43]:
import transformers
import torch
import math
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from mcllm.model.llm import *

In [44]:
def create_low_rank_matrix(m, n, rank):
    A = np.random.randn(m, rank) @ np.random.randn(rank, n)
    return A


mat = create_low_rank_matrix(100, 100, rank=3)

In [45]:
seq_len = 20
llm = NanoBERT(vocab_size=100, max_seq_len=20)

In [46]:
batch_size = 1
x = torch.LongTensor(np.zeros((batch_size, seq_len)))
out = llm(x)
out.shape

torch.Size([1, 20, 3])